# Build connection, send data to database

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://gcn.com/articles/2017/12/05/-/media/GIG/GCN/Redesign/Articles/2017/December/flightdata.png", width = 800)

**In this notebook, we will**
1. Set up a connection to a SQL database 
2. Download a csv file
3. Prepare the data for further processing
4. Push the prepared data to a table in the database
5. Use SQL to query data from the database



In [2]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import requests
from zipfile import *
from configdef import *
from sqlalchemy import exc #SQLAlchemy provides a nice “Pythonic” way of interacting with databases.
from sqlalchemy import event

We will go through this workflow step by step before we include all single steps into one, big, main function which does all the steps at once.  
For our puropse, we will use public **flight data**. 

# 1. Set up a connection to a SQL database

We start with connecting to an existing sql database, so that we can check what is already there and to send data to a table in the database later on.

In [3]:
# Establish db connection

# Get connection details from configdef file into a list
params = config(section='postgres')

# Use sql alchemy to create connection to database, which is contained within the engine object
engine = pg_engine_connection(**params)

# Cleans up unnecessary database connections
engine.dispose()

Postgres Database connection successful


Now, you can query data from a table in the database you are connected to.  
We are working with the table called nyflights, which contains data from 2019 and 2020 of the three New York airports (JFK, LGA and EWR).  
Get an overview of this table by querying the top three rows.

With ```engine.execute('sql query')``` you create an object:

In [12]:
engine.execute('select * from nyflights limit 3')

Using ```.fetchall()``` you can access the data you asked for in your query:

In [11]:
result = engine.execute('select * from nyflights limit 3').fetchall()
display(result)

[(2019, 10, 23, datetime.datetime(2019, 10, 23, 0, 0), 931.0, 936, -5.0, 1147.0, 1152, -5.0, '9E', 'N907XJ', 3475, 'LGA', 'TYS', 103.0, 648.0, 0, 0),
 (2019, 10, 24, datetime.datetime(2019, 10, 24, 0, 0), 943.0, 936, 7.0, 1151.0, 1153, -2.0, '9E', 'N606LR', 3475, 'LGA', 'TYS', 101.0, 648.0, 0, 0),
 (2019, 10, 25, datetime.datetime(2019, 10, 25, 0, 0), 935.0, 936, -1.0, 1144.0, 1153, -9.0, '9E', 'N914XJ', 3475, 'LGA', 'TYS', 105.0, 648.0, 0, 0)]

You see three rows each representing a single flight with different information in each column.

# 2. Download csv file

In the following, you are going to download a csv file containing additional flight data from [this website](https://transtats.bts.gov).    
You can specify, which data you want to download.  
We want you to choose a specific month in a specific year for your download.  
In order to avoid everybody's downloading the same data, first check which months are already available in the nyflights table:

In [13]:
#First, check which months are not in the database yet.
months = engine.execute('select distinct(year, month) from nyflights').fetchall()
months

[('(2018,9)',),
 ('(2019,1)',),
 ('(2019,2)',),
 ('(2019,3)',),
 ('(2019,4)',),
 ('(2019,5)',),
 ('(2019,6)',),
 ('(2019,7)',),
 ('(2019,8)',),
 ('(2019,9)',),
 ('(2019,10)',),
 ('(2019,11)',),
 ('(2019,12)',),
 ('(2020,1)',),
 ('(2020,2)',),
 ('(2020,3)',),
 ('(2020,4)',),
 ('(2020,5)',),
 ('(2020,6)',),
 ('(2020,7)',),
 ('(2020,8)',),
 ('(2020,9)',),
 ('(2020,10)',),
 ('(2020,11)',),
 ('(2020,12)',)]

Now, choose a month/year that is not yet in the database.  
With the following command lines, you will download a csv file on public flight data from [this website](https://transtats.bts.gov) containing data of your chosen month/year.    
The file will be stored in a data folder.

In [14]:
years = [2018] # list of years you want to look at, specify one year
months = [9] # list of months you want to look at, specify one month
# Here: September 2018 which is in the list already
path ='data/' # Specifies path for saving file

# Loop through months
for year in years:
    for month in months:
        # Get the file from the website https://transtats.bts.gov
        zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
        csv_file = f'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_{year}_{month}.csv'
        url = (f'https://transtats.bts.gov/PREZIP/{zip_file}')
        arg = f' -P {path} --no-check-certificate'
        # Use the wget Module to download the file. The method accepts two parameters: the URL path of the file to download and local path where the file is to be stored.
        !wget {url}{arg} 

--2021-03-29 14:28:00--  https://transtats.bts.gov/PREZIP/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2018_1.zip
Resolving transtats.bts.gov... 204.68.194.70
Connecting to transtats.bts.gov|204.68.194.70|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 29186677 (28M) [application/x-zip-compressed]
Saving to: 'data/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2018_1.zip'

On_Time_Reporting_C 100%[===================>]  27.83M   482KB/s    in 87s     

2021-03-29 14:29:28 (327 KB/s) - 'data/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2018_1.zip' saved [29186677/29186677]



In [32]:
# Unzip your file
with ZipFile(path+zip_file, 'r') as zip_ref:
    zip_ref.extractall(path)
    
# In casee this does not work for you try:
# Instead of 'from zipfile import *' use 'import zipfile' and use 'with zipfile.ZipFile(path+zip_file, 'r')'

In [33]:
# Read in your data
df = pd.read_csv(path+csv_file, low_memory = False)
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2018,1,1,27,6,2018-01-27,UA,19977,UA,N26232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,1,1,27,6,2018-01-27,UA,19977,UA,N477UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,1,1,27,6,2018-01-27,UA,19977,UA,N13720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,1,1,27,6,2018-01-27,UA,19977,UA,N16217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,1,1,27,6,2018-01-27,UA,19977,UA,N33714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df.shape

(570118, 110)

# 3. Prepare the csv file for further processing

In the next step, we clean and prepare our dataset.

a) Since the dataset consists of a lot of columns, we we define which ones to keep.

In [31]:
# Columns from downloaded file that are to be kept

columns_to_keep = [
                'Year',
                'Month',
                'DayofMonth',
                'FlightDate',
                'DepTime',
                'CRSDepTime',
                'DepDelay',
                'ArrTime',
                'CRSArrTime',
                'ArrDelay',
                'Reporting_Airline',
                'Tail_Number',
                'Flight_Number_Reporting_Airline',
                'Origin',
                'Dest',
                'AirTime',
                'Distance',
                'Cancelled',
                'Diverted'
]

In [28]:
# The columns in the DB have different naming as in the source csv files. Lets get the names from the DB
table_name_sql = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'nyflights' ORDER BY ordinal_position"
c_names = engine.execute(table_name_sql).fetchall()
c_names

[('year',),
 ('month',),
 ('day',),
 ('flightdate',),
 ('dep_time',),
 ('sched_dep_time',),
 ('dep_delay',),
 ('arr_time',),
 ('sched_arr_time',),
 ('arr_delay',),
 ('carrier',),
 ('tailnum',),
 ('flight',),
 ('origin',),
 ('dest',),
 ('air_time',),
 ('distance',),
 ('cancelled',),
 ('diverted',)]

In [26]:
# we can clean up the results into a clean list
new_column_names=[]
for name in c_names:
    new_column_names.append(name[0])
new_column_names        

['year',
 'month',
 'day',
 'flightdate',
 'dep_time',
 'sched_dep_time',
 'dep_delay',
 'arr_time',
 'sched_arr_time',
 'arr_delay',
 'carrier',
 'tailnum',
 'flight',
 'origin',
 'dest',
 'air_time',
 'distance',
 'cancelled',
 'diverted']

b) With the next function, we make our csv file ready to be uploaded to SQL.  
We only keep to above specified columns and convert the datatypes.

In [29]:
def clean_airline_df(df):
    '''
    Transforms a df made from BTS csv file into a df that is ready to be uploaded to SQL
    Set rows=0 for no filtering
    '''

    # Build dataframe including only the columns you want to keep
    df_airline = df.loc[:,columns_to_keep]
     
    # Clean data types and NULLs
    df_airline['Year']= pd.to_numeric(df_airline['Year'], downcast='integer')
    df_airline['Month']= pd.to_numeric(df_airline['Month'], downcast='integer')
    df_airline['DayofMonth']= pd.to_numeric(df_airline['DayofMonth'], downcast='integer')
    df_airline['FlightDate']= pd.to_datetime(df_airline['FlightDate'], yearfirst=True)
    df_airline['CRSArrTime']= pd.to_numeric(df_airline['CRSArrTime'], downcast='integer', errors='coerce')
    df_airline['Cancelled']= pd.to_numeric(df_airline['Cancelled'], downcast='integer')
    df_airline['Diverted']= pd.to_numeric(df_airline['Diverted'], downcast='integer')
    
    # Rename columns
    df_airline.columns = new_column_names
    
    return df_airline

In [35]:
# Call function and check resulting dataframe
df_clean = clean_airline_df(df)
df_clean.head()

,year,month,day,flightdate,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,cancelled,diverted
0,2018,1,27,2018-01-27,602.0,615,-13.0,756.0,808,-12.0,UA,N26232,369,FLL,IAH,148.0,966.0,0,0
1,2018,1,27,2018-01-27,614.0,618,-4.0,813.0,831,-18.0,UA,N477UA,368,SEA,SFO,98.0,679.0,0,0
2,2018,1,27,2018-01-27,828.0,830,-2.0,1108.0,1107,1.0,UA,N13720,367,DCA,IAH,190.0,1208.0,0,0
3,2018,1,27,2018-01-27,641.0,650,-9.0,1242.0,1250,-8.0,UA,N16217,366,LAX,ORD,212.0,1744.0,0,0
4,2018,1,27,2018-01-27,1810.0,1824,-14.0,2021.0,2045,-24.0,UA,N33714,365,JAX,EWR,112.0,820.0,0,0


If you decide to only look at specific airports, it is a good decision to filter for them in advance.  
This function does the filtering. 

In [36]:
# Specify the airports you are interested in and put them as a list in the function.
def select_airport(df, airports):
    ''' Helper function for filtering airline df for a subset of airports'''
    df_out = df.loc[(df.origin.isin(airports)) | (df.dest.isin(airports))]
    return df_out

In [37]:
# Execute function, not filtering for any specific airports
airports=[]
if len(airports) > 0:
    df_airline = select_airport(df_clean, airports)
else:
    df_airline = df_clean
    
df_airline.head()

,year,month,day,flightdate,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,cancelled,diverted
0,2018,1,27,2018-01-27,602.0,615,-13.0,756.0,808,-12.0,UA,N26232,369,FLL,IAH,148.0,966.0,0,0
1,2018,1,27,2018-01-27,614.0,618,-4.0,813.0,831,-18.0,UA,N477UA,368,SEA,SFO,98.0,679.0,0,0
2,2018,1,27,2018-01-27,828.0,830,-2.0,1108.0,1107,1.0,UA,N13720,367,DCA,IAH,190.0,1208.0,0,0
3,2018,1,27,2018-01-27,641.0,650,-9.0,1242.0,1250,-8.0,UA,N16217,366,LAX,ORD,212.0,1744.0,0,0
4,2018,1,27,2018-01-27,1810.0,1824,-14.0,2021.0,2045,-24.0,UA,N33714,365,JAX,EWR,112.0,820.0,0,0


# 4. Push the prepared data to a table in the database

In [38]:
# Specify which table within your database you want to push your data to. For our case, use the nyflights table.
# If the specified table doesn't exist yet, it will be created
# With 'append', your data will be appended to the already existing data within the table nyflights.
# This will take a minute or two...

table = 'nyflights'

# Sends your data to specified table, via the etablished connection
df_airline.to_sql(table, engine, index=False, if_exists="append", 
    method='multi', chunksize=5000)
print(f'done uploading {year}-{month}')

done uploading 2018-1


# 5. Use SQL to query data from the database

### Tasks
Having sent the data to the database, it is always good to check the results.

1. Check the top 5 rows of the nyflights table (as we did above)

2. Check if your data has been added by selecting your month and year.

3. See how many rows you have sent to the database by selecting your month and year in combination with count and group by.

# 6. Wrap-up

The following function puts everything we did above together.  

In [41]:
def airline_csv_to_sql(years=[2020], months=[2], path ='data/', table='flights', airports=[]):
    '''Downloads and unzips the flight data from BTS'''
    
    # Establish db connection
    params = config(section='postgresql')
    engine = pg_engine_connection(**params)
    engine.dispose()

    # Loop through months
    for year in years:
        for month in months:
            # Get the file
            zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
            csv_file = f'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_{year}_{month}.csv'
            url = (f'https://transtats.bts.gov/PREZIP/{zip_file}')
            arg = f' -P {path} --no-check-certificate'
            !wget {url}{arg} 

            # unzip
            with ZipFile(path+zip_file, 'r') as zip_ref:
                zip_ref.extractall(path)
            
            # prepare df
            df_airline = pd.read_csv(path+csv_file, low_memory=False)
            df_airline = clean_airline_df(df_airline)
            
            # Select specific airports 
            if len(airports) > 0:
                df_airline = select_airport(df_airline, airports)

            # to SQL
            print(f'starting uploading {df_airline.shape[0]} rows from {year}-{month}')
            df_airline.to_sql(table, engine, index=False, if_exists="append", 
                  method='multi', chunksize=5000)
            print(f'done uploading {year}-{month}')